# [prompts](https://python.langchain.com/docs/modules/model_io/prompts/)
语言模型的提示是由用户提供的一组指令或输入，用于指导模型的响应，帮助它理解上下文并生成相关且一致的基于语言的输出，例如回答问题、完成句子或参与对话。



In [2]:
from dotenv import load_dotenv, find_dotenv
from langchain.globals import set_debug
load_dotenv(find_dotenv())
set_debug(False)

## 示例
这个快速入门提供了如何使用提示的基本概述。

提示模板是为语言模型生成提示的预定义配方。

模板可能包括说明、少量示例以及适合给定任务的特定上下文和问题。

LangChain提供了创建和使用提示模板的工具。

LangChain努力创建与模型无关的模板，以便在不同的语言模型之间轻松重用现有模板。

通常，语言模型期望提示符要么是字符串，要么是聊天消息列表。

### PromptTemplate
使用`PromptTemplate`为字符串提示创建一个模板。默认情况下，`PromptTemplate`使用Python的`str.format`语法进行模板化。

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=["adjective", "content"], 
                                 template= "Tell me a {adjective} joke about {content}.")

prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="funny", content="chickens")


'Tell me a funny joke about chickens.'

模板支持任意数量的变量，包括没有变量

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

### [ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)
提示聊天模型/是聊天消息的列表。

每个聊天消息与内容相关联，还有一个称为角色的附加参数。例如，在OpenAI聊天完成API中，聊天消息可以与AI助手，人类或系统角色相关联。

In [6]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")

将这些格式化的消息导入到LangChain的ChatOpenAI聊天模型类中，大致相当于直接使用OpenAI客户端的以下操作

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful AI bot. Your name is Bob."},
        {"role": "user", "content": "Hello, how are you doing?"},
        {"role": "assistant", "content": "I'm doing well, thanks!"},
        {"role": "user", "content": "What is your name?"},
    ],
)

`ChatPromptTemplate.from_messages`接受各种消息表示，并且是一种方便的方法，可以用您想要的消息格式化聊天模型的输入。

例如，除了使用上面使用的(类型、内容)的二元组表示之外，您还可以传入`MessagePromptTemplate`或`BaseMessage`的实例。

In [7]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


### Message Prompts
LangChain提供了不同类型的`MessagePromptTemplate`。最常用的是`AIMessagePromptTemplate`, `SystemMessagePromptTemplate`和`HumanMessagePromptTemplate`，它们分别创建AI消息，系统消息和人类消息。您可以在这里阅读有关[不同类型消息的更多信息](https://python.langchain.com/docs/modules/model_io/chat/message_types/)。


在聊天模型支持使用任意角色接收聊天消息的情况下，您可以使用`ChatMessagePromptTemplate`，它允许用户指定角色名称。

In [8]:
from langchain_core.prompts import ChatMessagePromptTemplate

prompt = "May the {subject} be with you"

chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role="Jedi", template=prompt
)
chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you', role='Jedi')

### MessagesPlaceholder
LangChain还提供了`MessagesPlaceholder`，它使您可以完全控制在格式化期间呈现哪些消息。当您不确定应该为消息提示模板使用什么角色，或者希望在格式化过程中插入消息列表时，此功能非常有用。

In [14]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), human_message_template]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='Summarize our conversation so far in {word_count} words.'))])

In [11]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(
    content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

chat_prompt.format_prompt(
    conversation=[human_message, ai_message], word_count="10"
).to_messages()

[HumanMessage(content='What is the best way to learn programming?'),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience'),
 HumanMessage(content='Summarize our conversation so far in 10 words.')]

In [16]:
# prompt = MessagesPlaceholder("history")
# prompt.format_messages() # raises KeyError

prompt = MessagesPlaceholder("history", optional=True)
prompt.format_messages() # returns empty list []

prompt.format_messages(
    history=[
        ("system", "You are an AI assistant."),
        ("human", "Hello!"),
    ]
)

[SystemMessage(content='You are an AI assistant.'),
 HumanMessage(content='Hello!')]

### LCEL
`PromptTemplate`和`ChatPromptTemplate`实现了`Runnable`接口，这是`LangChain`表达式语言(LCEL)的基本构建块。这意味着它们支持`invoke`, `ainvoke`, `stream`, `aststream`, `batch`, `abbatch`, `aststream`日志调用。

`PromptTemplate`接受一个字典(提示变量的字典)并返回一个`StringPromptValue`。`ChatPromptTemplate`接受一个字典并返回一个`ChatPromptValue`。

In [17]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
prompt_val

StringPromptValue(text='Tell me a funny joke about chickens.')

In [18]:
prompt_val.to_string()

'Tell me a funny joke about chickens.'

In [19]:
prompt_val.to_messages()

[HumanMessage(content='Tell me a funny joke about chickens.')]

In [21]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
chat_val

ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')])

In [22]:
chat_val.to_messages()

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
 HumanMessage(content='i dont like eating tasty things.')]

In [23]:
chat_val.to_string()

"System: You are a helpful assistant that re-writes the user's text to sound more upbeat.\nHuman: i dont like eating tasty things."

## [例选择器](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/)


如果您有大量的示例，您可能需要选择在提示符中包含哪些示例。示例选择器是负责这样做的类。

基接口定义如下:


In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, List, Any
class BaseExampleSelector(ABC):
    """Interface for selecting examples to include in prompts."""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Select which examples to use based on the inputs."""
        
    @abstractmethod
    def add_example(self, example: Dict[str, str]) -> Any:
        """Add new example to store."""

它需要定义的唯一方法是一个`select_examples`方法。它接受输入变量，然后返回一个示例列表。如何选择这些示例取决于每个具体实现。

`LangChain`有几种不同类型的示例选择器。有关所有这些类型的概述，请参见下表。

在本指南中，我们将逐步创建一个自定义示例选择器。


### Examples

为了使用示例选择器，我们需要创建一个示例列表。这些通常应该是示例输入和输出。出于演示目的，让我们假设我们正在选择如何将英语翻译成意大利语的示例。

In [26]:
examples = [
    {"input": "hi", "output": "ciao"},
    {"input": "bye", "output": "arrivaderci"},
    {"input": "soccer", "output": "calcio"},
]

### 自定义示例选择器

让我们编写一个示例选择器，它根据单词的长度选择要选择的示例。

In [27]:
from langchain_core.example_selectors.base import BaseExampleSelector


class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        # This assumes knowledge that part of the input will be a 'text' key
        new_word = input_variables["input"]
        new_word_length = len(new_word)

        # Initialize variables to store the best match and its length difference
        best_match = None
        smallest_diff = float("inf")

        # Iterate through each example
        for example in self.examples:
            # Calculate the length difference with the first word of the example
            current_diff = abs(len(example["input"]) - new_word_length)

            # Update the best match if the current one is closer in length
            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match]

In [33]:
example_selector = CustomExampleSelector(examples)
example_selector

In [28]:
example_selector = CustomExampleSelector(examples)
example_selector.select_examples({"input": "okay"})

[{'input': 'bye', 'output': 'arrivaderci'}]

In [29]:
example_selector.add_example({"input": "hand", "output": "mano"})
example_selector.select_examples({"input": "okay"})

[{'input': 'hand', 'output': 'mano'}]

In [31]:
example_selector

### Use in a Prompt
现在我们可以在提示符中使用这个示例选择器

In [34]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

example_prompt

PromptTemplate(input_variables=['input', 'output'], template='Input: {input} -> Output: {output}')

In [36]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="Translate the following words from English to Italain:",
    input_variables=["input"],
)

print(prompt.format(input="wordss"))

Translate the following words from English to Italain:

Input: soccer -> Output: calcio

Input: wordss -> Output:


### Example Selector Types
| 名称 | 描述 |
| ---------------- | ----------- |
| 相似性 | 使用输入和示例之间的语义相似性来决定选择哪个示例。|
| MMR | 使用输入和示例之间的最大边际相关性来决定选择哪个示例。|
| Length | 选择基于多少可以适合在一定长度的例子|
| Ngram | 使用输入和示例之间的ngram重叠来决定选择哪个示例。|

#### Select by length

这个示例选择器根据长度选择要使用的示例。当您担心构造一个将超过上下文窗口长度的提示时，这很有用。对于较长的输入，它会选择更少的例子，而对于较短的输入，它会选择更多的例子。

In [37]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25,
    # The function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [38]:
# An example with small input, so it selects all examples.
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


In [39]:
# An example with long input, so it selects only one example.
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


In [40]:
# You can add an example to an example selector as well.
new_example = {"input": "big", "output": "small"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="enthusiastic"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output: small

Input: enthusiastic
Output:
